In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

/opt/anaconda3/envs/ai574-pocs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0622 20:52:43.731000 9501 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


# Load model
https://huggingface.co/luqh/ClinicalT5-large

In [2]:
MODEL_ID = "luqh/ClinicalT5-large"
tokenizer = T5Tokenizer.from_pretrained(MODEL_ID, model_max_length = 1024)
model = T5ForConditionalGeneration.from_pretrained(MODEL_ID, from_flax = True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/anaconda3/envs/ai574-pocs/lib/python3.11/site-packages/transformers/modeling_flax_pytorch_utils.py:460: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppr

# Some medical text

Not that this will fail if there are too many tokens. We'll have to do some chunking or RAG in order to
deal with the small context window.

In [14]:
medical_text = """
Non-steroidal anti-inflammatory drugs are not only potent analgesics and antipyretics but also nephrotoxins, and may cause 
electrolyte disarray. In addition to the commonly expected effects, including hyperkalemia, hyponatremia, acute renal injury, 
renal cortical necrosis, and volume retention, glomerular disease with or without nephrotic syndrome or nephritis can occur as 
well including after years of seemingly safe administration. Minimal change disease, secondary membranous glomerulonephritis, 
and acute interstitial nephritis are all reported glomerular lesions seen with non-steroidal anti-inflammatory use. We report a 
patient who used non-steroidal anti-inflammatory drugs for years without diabetes, chronic kidney disease, or proteinuria; he 
then developed severe nephrotic range proteinuria with 7 g of daily urinary protein excretion. Renal biopsy showed minimal 
change nephropathy, a likely secondary membranous glomerulonephritis, and acute interstitial nephritis present simultaneously
in one biopsy. 

"""

In [15]:
len(medical_text)

1030

# Inference

In [18]:
prompt = medical_text.strip()
inputs = tokenizer(prompt, return_tensors = "pt", truncation = False)

with torch.no_grad():
    summary_ids = model.generate(**inputs,
        max_new_tokens = 64, num_beams = 4, length_penalty = 1.5, early_stopping = False, no_repeat_ngram_size = 3, encoder_no_repeat_ngram_size = 4)

In [19]:
for summary_id in summary_ids:
    summary = tokenizer.decode(summary_id, skip_special_tokens = True)
    print(summary)

with minimal change disease in one biopsy and minimal change glomerulopathy in another biopsy. renal biopsy was negative for any of these lesions. renal biopsy also showed minimal change renal disease, renal biopsy showed no evidence of acute renal failure.. non-. non-
